In [2]:
import numpy as np
import pandas as pd
import csv_figures
from csv_figures import add_time_col
from csv_figures import annual_Norm
import matplotlib.pyplot as plt
import requests
from matplotlib.widgets import CheckButtons

In [3]:
fort_yukon = pd.read_excel('../../Cleaned_data/RuralAk/RuralAK_Fort_Yukon_cleaned.xlsx')

In [4]:
fort_yukon_new = add_time_col(fort_yukon)

/Users/guge/Desktop/acep-solar/new/ACEP_solar/GEGU/figures/csv_figures.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dataframe['Date'][i] = str(dataframe['Year'][i]) + '-' + str(dataframe['Month'][i])


In [5]:
fort_annual_values,fort_month = annual_Norm(fort_yukon_new)

In [6]:
new_fort = pd.DataFrame({'Date':fort_month,'fort_yukon':fort_annual_values})

In [7]:
# Get the data from the PV Watts --TMY2
list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 18, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": 50, "azimuth": 180, "lat": 66.56, "lon": -145.25, "dataset": 'tmy2'}
json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
TMY2 = pd.DataFrame(data = json_response['outputs'])
# Get the data from the PV Watts --TMY2
list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 18, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": 50, "azimuth": 180, "lat": 66.56, "lon": -145.25, "dataset": 'tmy3'}
json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
TMY3 = pd.DataFrame(data = json_response['outputs'])

In [8]:
# TMY2_low = TMY2['ac_annual'][2]*0.95/80
# TMY2_high = TMY2['ac_annual'][2]*1.05/80

TMY3_low = TMY3['ac_annual'][2]*0.95/18
TMY3_high = TMY3['ac_annual'][2]*1.05/18

# new_fort['TMY2_low'] = TMY2_low
# new_fort['TMY2_high'] = TMY2_high
new_fort['TMY3_low'] = TMY3_low
new_fort['TMY3_high'] = TMY3_high
new_fort

,Date,fort_yukon,TMY3_low,TMY3_high
0,2016-07-01,659.664778,995.597423,1100.397152
1,2016-08-01,781.043111,995.597423,1100.397152
2,2016-09-01,871.573667,995.597423,1100.397152
3,2016-10-01,887.238111,995.597423,1100.397152
4,2016-11-01,938.127556,995.597423,1100.397152
5,2016-12-01,943.689778,995.597423,1100.397152
6,2017-01-01,943.756444,995.597423,1100.397152
7,2017-02-01,945.916667,995.597423,1100.397152
8,2017-03-01,941.527778,995.597423,1100.397152
9,2017-04-01,987.972222,995.597423,1100.397152


In [10]:
%matplotlib qt5
fig, ax = plt.subplots(figsize = (15, 8))
x = new_fort['Date']


y_fort = new_fort['fort_yukon']
l0, = ax.plot(x ,y_fort,label='Fort Yukon')

y_TMY3_low = new_fort['TMY3_low']
l1, = ax.plot(x,y_TMY3_low,'-.',label = 'TMY3_low')

y_TMY3_high = new_fort['TMY3_high']
l2, = ax.plot(x,y_TMY3_high,'-.',label = 'TMY3_high')

ax.set_title("Rolling 12-Month Average of Produced Power, Over Time", size = 18)
ax.set_xlabel("Month", size = 14)
ax.set_ylabel("AC Power Produced (kWh) per DC Power Installed (kW)", size = 14)
ax.legend()


plt.subplots_adjust(left=0.2)
lines = [l0, l1, l2]

# Make checkbuttons with all plotted lines with correct visibility
rax = plt.axes([0.005, 0.48, 0.15, 0.4])
labels = [str(line.get_label()) for line in lines]
visibility = [line.get_visible() for line in lines]
check = CheckButtons(rax, labels, visibility)


def func(label):
    index = labels.index(label)
    lines[index].set_visible(not lines[index].get_visible())
    plt.draw()

check.on_clicked(func)


# Some 

plt.show()

#Ok,this seems to be fine,maybe we will talk about it after.